In [ ]:
# run this cell first
from lamb.meta import Partial
lang.set_system(lang.td_presup)
lang.get_system()

# Presuppositions via partiality

The Heim & Kratzer approach to presuppositions works by allowing denotations and functions to be partially defined: the definedness conditions characterize the conditions under which the presupposition is satisfied.  Strictly speaking, it is *only* denotations and functions that involve partiality in this system, that is, not ordinary formulas (to the extent they exist in H&K).  However, it is impossible to implement this without having some representation for partiality in the metalanguage.  This is accomplished with the class `meta.Partial`.  

Unlike most subclasses of `TypedExpr`, instances of `Partial` do not have their own unique type -- their type is determined from their content.  Orthogonally to the type system, they layer a typed expression of type `t` on top of that content object, corresponding to the partiality.  They are drawn with two rows between bars.  The following example is the expression of type `e` that refers to `Joanna`, but has a definedness condition that that individual be salient.  In the strict Heim and Kratzer sense, such formulas don't really appear on their own, since definedness conditions like this need to be associated with some denotation, as in the second cell below.

In [ ]:
Partial(te("Joanna_e"), te("Salient_<e,t>(Joanna_e)"))

In [ ]:
%%lamb
||joanna|| = Partial(Joanna_e, Salient(Joanna_e))

The main case where you get partiality in a formula is for partially defined functions.  The lambbda notebook notation for this is basically the same: use `Partial` in the body of a function.  For example, if you take the noun `dog` to presuppose animacy of its argument, you would write that as follows:

In [ ]:
%%lamb
||dog|| = L x_e : Partial(Dog(x), Animate(x))

More examples (mixed with things I'll use later):

In [ ]:
%%lamb
||cat|| = L x_e : Cat(x)
#||the|| = L f_<e,t> : Partial((Iota x_e : f(x)), (ExistsExact x_e : f(x)))
||the|| = L f_<e,t> : IotaPartial x_e : f(x) & x << C_{e}
||everyone|| = L f_<e,t> : Partial((Forall x_e : Sentient(x) ==> f(x)), (Exists x_e : Sentient(x)))
||a|| = L f_<e,t> : L g_<e,t> : Exists x_e : f(x) & g(x)
||alfonso|| = Alfonso_e
||bark|| = L x_e : Partial(Bark(x), Canine(x))
||joanna|| = Partial(Joanna_e, Salient(Joanna_e))
testf = L f_<e,t> : L x_e : Partial(f(x), f(x))
||saw|| = L y_e : L x_e : Saw(x,y)
||POSS|| = L f_<e,<e,t>> : L x_e : IotaPartial y_e : f(x)(y)
||brother|| = L x_e : L y_e : BrotherOf(y,x)

Reduction of a partially defined function is reasonably straightforward: the variable is replaced as expected in both the ordinary body of the function and in the partiality condition.

In [ ]:
dog.content(alfonso.content).reduce()

Here's another simple case, where there's really only one presuppositional item in a derivation.  (Here I'm showing full composition directly, which I'll return to.)

In [ ]:
(alfonso * (saw * (alfonso * (POSS * brother))))

The above case is easy, but reduction with partiality gets complicated very quickly.  For example, what happens when the argument to a function is partial?  These are cases that as humans working with Heim & Kratzer pseudo-formulas in our head we can usually just get automatically, but they are not at all straightforward to think through mechanically.  Here is an example using the presupposition entry for `dog` above.  The partiality condition resulting from application gets lodged in a subformula under the existential quantifier, which actually binds into the partiality condition.  While this isn't necessarily the true analysis of `dog`, this situation might easily arise e.g. with relative clauses containing presupposition triggers.

In [ ]:
a.content(dog.content).reduce_all()

In [ ]:
a.content(dog.content)(bark.content).reduce_all()

Simplifying this requires some semantics for the metalanguage (i.e. a formal understanding of what partiality should amount to in expressions like this).  The projection problem to some degree rests on this as well.  What the lambda notebook implements for this composition system is a fairly strict notion of partiality (that sometimes does odd things).  It is important to be clear that this is **not** a three-valued logic, and will behave quite differently from the usual three-valued logics used for projection in a range of cases.

Another thing to know is that the lambda notebook will not fully simplify quantified formulas with partiality until beta reduction is complete: this is done because the possibility of reducing with arguments that are themselves partial greatly complicates the picture.

The result of projecting the partiality out of the subformula above can be seen by composing the three NL items together.  As a rule, this sort of inference is not automatically triggered in the metalanguage ever, but is always triggered by composition (following the Heim and Kratzer treatment of partiality, I would argue).  This could be forced in the metalanguage by calling `calculate_partiality()` on a `TypedExpr`.

Finally, it is useful to know that the composition operations, again following Heim & Kratzer, manipulate partiality directly. Outer partiality conditions from input expressions are removed from composing elements and conjoined to the output. By "outer", I mean if the expression composing is itself a `Partial`. Partiality is not calculated until after reduction, so functions do not typically have outer partiality conditions.

In [ ]:
(a * dog)

The above expression is not fully reduced, but when we supply `g`, it can be, and so partiality is projected. In this toy example, both `dog` and `bark` supply definitedness conditions, which end up existentially quantified:

In [ ]:
((a * dog) * bark)

The process by which partiality gets projected can be seen by looking at a derivation:

In [ ]:
((a * dog) * bark)[0].content.derivation

Things can get more complicated when presuppositions interact; here is an example composing `the` with the entry above for `dog`.  `The` is defined here using a version of $\iota$ that will always lead to an $\exists !$ partiality condition.

In [ ]:
(the * dog).tree()

Here's another example of (plausibly) this type of noun:

In [ ]:
%%lamb
||bachelor|| = L x_e : Partial(Unmarried(x) & Male(x) & Adult(x), Adult(x))

In [ ]:
the * bachelor

## Quantifiers

The interaction of quantifiers and partiality is generally somewhat complicated. At an intuitive level, a partiality condition in the scope of a quantifier that involves the variable bound by that quantifier will project a definedness condition that allows the quantified expression to be interpreted. It is worth noting at the outset, that this is not a *theory* of the interaction of natural language quantifiers and presuppositions, though it does provide tools for developing one. For example, assuming the presuppositional entry above for "bachelor", consider the sentence, *Everyone is a bachelor*. This involves the formula $Adult(x)$ in the scope of a $\forall x$. To ensure that this formula is interpretable, the semantics of `Partial` will require that every element in the domain meet this presupposition. (See below for ways to weaken this, and the appendix for more cases.)

In [ ]:
(everyone * bachelor)[0].content.derivation# "everyone is a bachelor"

In [ ]:
(everyone * (saw * joanna)).tree()

In [ ]:
# everyone saw everyone
# note: the result here can be simplified (but is beyond the scope of lamb's simplification code)! It doesn't presuppose two distinct sentient individuals.
(everyone * (lang.Binder(1) * (everyone * (lang.Binder(2) * (lang.Trace(1) * (saw * lang.Trace(2)))))))

Next I give some more complicated examples involving binding of possessive pronouns:

    (1) Joanna likes her_5 brother.
    (2) Every_6 student likes his_6 brother.
    (3) Every_6 student who_5 likes his_5 brother likes his_6 sister.
  
The resulting presuppositions are quite complicated, involving multiple layers of definedness. When a `Partial` is embedded in a `Partial` condition, the result is defined only if the embedded `Partial` is both defined and true, and can't be further simplified (without model theoretic interpretation).

To start, here is how to define indexed presupposition pronouns:

In [ ]:
male_prop = %te L x_e : Male_<e,t>(x)
female_prop = %te L x_e : Female_<e,t>(x)
himi = lang.PresupPronoun.index_factory("him", male_prop)
heri = lang.PresupPronoun.index_factory("her", female_prop)
himi(5)

Side-note: notice that binding a pronoun of this sort always generates a partial function.

In [ ]:
%%lamb
||joanna|| = Partial(Joanna_e, Salient(Joanna_e))
||student|| = L x_e : Student(x)
||likes|| = L x_e : L y_e : Likes(y,x)
||brother|| = L x_e : L y_e : BrotherOf(y,x)
||sister|| = L x_e : L y_e : SisterOf(y,x)
||every|| = L f_<e,t> : L g_<e,t> : Partial((Forall x_e : f(x) ==> g(x)), (Exists x_e : f(x)))

||POSS|| = L f_<e,<e,t>> : L x_e : IotaPartial y_e : f(x)(y)

In [ ]:
(joanna * (lang.Binder(5) * (lang.Trace(5) * (likes * (heri(5) * (POSS * brother))))))

The result of this composition involves a layered sequence of definedness conditions. The whole thing is defined just in case Joanna is salient, and Joanna is female, and there is exactly one y that is a brother of Joanna, in that order. In this case, nothing would be obviously harmed by any sort of reordering, but in the next example we will see a case where sequencing of definedness conditions matters for interpretation.

To handle the next case, we will need to apply some advanced techniques to weaken the behavior of universal quantification. Natural language determiners have a tripartite structure, and the presuppositions of the scope do not project absolutely. The exact empirical pattern is somewhat debated in the literature, but a minimal assumption is that presuppositions of the scope do not apply to entities that do not meet the restrictor condition. One way to capture this idea is to use a version of $\rightarrow$ from three-valued logics; here in particular we use the Kleene arrow operator.

This is implemented via the metalanguage operators `Body` and `Condition`, which can be used to explicitly split parts of `Partial` expressions. These operators do not require a `Partial`. `Body(x)` where `x` is not a `Partial` just gives `x`, and `Condition(x)` gives `True`.

In [ ]:
%%lamb
karrow = L p_t : L q_t : Partial((p >> Body(q)), (p >> Condition(q)))
||every|| =<reduce> L f_<e,t> : L g_<e,t> : Forall x_e : karrow(f(x))(g(x))

In [ ]:
himi(5) * (POSS * brother) # "his_5 brother"

In [ ]:
((every * student) * (lang.Binder(5) * ((lang.Trace(5) * (likes * (himi(5) * (POSS * brother))))))) # "every_5 student likes his_5 brother"

A prediction of this account is that every student is presupposed to be male, which may or may not be empirically adequate.

In [ ]:
((every * student) * (lang.Binder(5) * ((lang.Trace(5) * (likes * (himi(5) * (POSS * brother))))))).tree()

If there are presuppositions introduced in the restrictor, the above may *still* not be sufficient. For example:

    (3) Every_6 student who_5 likes his_5 brother likes his_6 sister.

In (3), we do not want to presuppose that every entity is male; the previous entry for `every` would lead to this presupposition. One way to handle this via partiality is to build a multi-layered presupposition directly in the lexical entry. In the example entry below, the inner definedness condition is that every entity in the domain satisfies the restrictor presupposition. If this is satisfied, then there is an outer definedness condition that every entity satisfying the restrictor satisfies the scope presuppositions. If both of these are satisfied, then the main at-issue implication is defined. The inner definedness condition is arguably still too empirically strong, but for the sake of documentation, I will leave it here.

In [ ]:
%%lamb
||every|| = L f_<e,t> : L g_<e,t> : Forall x_e : Partial(((x << C_{e} & Body(f(x))) >> Body(g(x))), Partial(((x << C_{e} & Body(f(x))) >> Condition(g(x))), ((x << C_{e}) >> Condition(f(x)))))


In [ ]:
student * (lang.Binder(5) * ((lang.Trace(5) * (likes * (himi(5) * (POSS * brother))))))

In [ ]:
# compute the subject: "[every student [5 [that t_5 likes his_5 brother]]]"
subject = (every * (student * (lang.Binder(5) * ((lang.Trace(5) * (likes * (himi(5) * (POSS * brother))))))))
subject

The ultimate result is very complex:

In [ ]:
# assume that the subject has QR'd in order to bind the possessive pronoun
subject * (lang.Binder(6) * (lang.Trace(6) * (likes * (himi(6) * (POSS * sister)))))

### Appendix: calculate_partiality reference

This appendix presents the core partiality semantics (triggered by calling `calculate_partiality()`) for quick reference.

In [ ]:
reload_lamb()

First, a few notes about the basic logic of `Partial` expressions. A `Partial` with condition `True` is unconditionally equivalent to its body.

In [ ]:
te("Partial(True, True)").calculate_partiality()

In [ ]:
te("Partial(False, True)").calculate_partiality()

A `Partial` with condition `False` is uniformly uninterpretable, and such `Partial`s (though they store the condition for display purposes) are equivalent to each other at the body's type. Keep in mind that they do not have a value at that type, however!

In [ ]:
te("Partial(True, False)").calculate_partiality()

In [ ]:
te("Partial(False, False)").calculate_partiality()

In [ ]:
te("Partial(True, False)") == te("Partial(False, False)")

In [ ]:
te("Partial(True, False)") == te("Partial(False, Partial(True, False))")

It is important to note at the outset that this system is *not* intended as one of the standard three-valued logic. An expression with a failed `Partial` at any point in it is equivalent to a failed `Partial` at the relevant type, no matter the logical operators involved. E.g. in Strong Kleene, `False & Undef` would come out false, but here it comes out undefined.

In [ ]:
te("Partial(True, True) & Partial(True, False)").calculate_partiality().derivation

In [ ]:
te("Partial(False, True) & Partial(True, False)").calculate_partiality().derivation

From here we move on to binding operators. There are two modes for calculating partiality with variable-binding expressions. They are best thought of in terms of the model theory.  In *weak* mode, the projected partiality condition to the domain is the minimum to support the truth or falsity of the quantified body, with the same value for the variable in both the body and the condition. In several cases, the body will also need to incorporate the condition.  In *strict* mode, the entire domain must satisfy the condition for the quantification to get off the ground. Strict mode is easier to understand, and probably more natural from the model-theoretic perspective, but unfortunately it isn't very useful in practical terms because the projected partiality conditions are very strong.

In both systems, lambda operators will block projection until they have been eliminated by beta-reduction, and variables bound by lambda functions will also block projection.  The latter is to enforce a reduction $\gg$ partiality-calculation order, which is not necessary in principle but greatly aids in simplification.

The next set of cells illustrate the weak projection semantics, where the operator in question is duplicated in both the condition and the body. 

In [ ]:
meta.BindingOp.partiality_weak = True # default

In [ ]:
te("Forall x_e : Partial(P_<e,t>(x), Q_<e,t>(x))").calculate_partiality()

In [ ]:
te("Exists x_e : Partial(P_<e,t>(x), Q_<e,t>(x))").calculate_partiality()

In [ ]:
te("Lambda x_e : Partial(P_<e,t>(x), Q_<e,t>(x))").calculate_partiality()

In [ ]:
te("ExistsExact x_e : Partial(P_<e,t>(x), Q_<e,t>(x))").calculate_partiality()

In [ ]:
te("ExistsExact x_e : P_<e,t>(x)").calculate_partiality()

In [ ]:
te("Iota x_e : Partial(P_<e,t>(x), Q_<e,t>(x))").calculate_partiality()

In [ ]:
te("Iota x_e : P_<e,t>(x)").calculate_partiality()

The `IotaPartial` class ensures an exactness partiality condition.

In [ ]:
te("IotaPartial x_e : Partial(P_<e,t>(x), Q_<e,t>(x))").calculate_partiality()

In [ ]:
te("IotaPartial x_e : P_<e,t>(x)").calculate_partiality()

In [ ]:
# it's unclear whether this is right, but it is based on Lambda.
te("Set x_e : Partial(P_<e,t>(x), Q_<e,t>(x))").calculate_partiality()

In [ ]:
te("A_e << (Set x_e : Partial(P_<e,t>(x), Q_<e,t>(x)))").reduce_all().calculate_partiality()

The next set of cells illustrate the strong partiality projection semantics, where bound conditions must be true at the entire domain.

In [ ]:
reload_lamb()
meta.BindingOp.partiality_weak = False

In [ ]:
te("Forall x_e : Partial(P_<e,t>(x), Q_<e,t>(x))").calculate_partiality()

In [ ]:
te("Exists x_e : Partial(P_<e,t>(x), Q_<e,t>(x))").calculate_partiality()

In [ ]:
te("Lambda x_e : Partial(P_<e,t>(x), Q_<e,t>(x))").calculate_partiality()

In [ ]:
te("ExistsExact x_e : Partial(P_<e,t>(x), Q_<e,t>(x))").calculate_partiality()

In [ ]:
te("ExistsExact x_e : P_<e,t>(x)").calculate_partiality()

In [ ]:
te("Iota x_e : Partial(P_<e,t>(x), Q_<e,t>(x))").calculate_partiality()

In [ ]:
te("Iota x_e : P_<e,t>(x)").calculate_partiality()

In [ ]:
te("IotaPartial x_e : Partial(P_<e,t>(x), Q_<e,t>(x))").calculate_partiality()

In [ ]:
te("IotaPartial x_e : P_<e,t>(x)").calculate_partiality()

In [ ]:
# it's unclear whether this is right, but it is based on Lambda.
te("Set x_e : Partial(P_<e,t>(x), Q_<e,t>(x))").calculate_partiality()

In [ ]:
te("A_e << (Set x_e : Partial(P_<e,t>(x), Q_<e,t>(x)))").reduce_all().calculate_partiality()